In [1]:
import os
from datetime import datetime 
from datetime import timezone

In [2]:
from tabulate import tabulate
import netCDF4
from pystac import ItemCollection, Item, Asset

In [3]:
input_stac_collection = 'test/stage_in/stage_in_results.json' # type: stage-in
output_stac_collection = 'process_results.json' # type: stage-out

# Filename written to the working directory
summary_table_filename = "summary_table.txt"

# Examples of arbitrary arguments of different data types
example_argument_int = 1
example_argument_float = 1.0
example_argument_string = "string"
example_argument_bool = True
example_argument_empty = None

# Output Example Arguments

Table useful for debugging connection of arguments from CWL into notebook. Used for unity-app-generator development and as an example, not an application requirement.

In [4]:
argument_names = [
    'example_argument_int',
    'example_argument_float',
    'example_argument_string',
    'example_argument_bool',
    'example_argument_empty',
]

table_data = []
column_names = ['argument_name', 'type', 'value']
for arg_name in argument_names:
    arg_val = locals()[arg_name]
    table_data.append( (arg_name, type(arg_val), arg_val) )

tabulate(table_data, headers=column_names, tablefmt='html')

argument_name,type,value
example_argument_int,<class 'int'>,1
example_argument_float,<class 'float'>,1.0
example_argument_string,<class 'str'>,string
example_argument_bool,<class 'bool'>,True
example_argument_empty,<class 'NoneType'>,


# Import Files from STAC Item Collection

Load filenames from the stage_in STAC item collection file

In [5]:
import json

with open(input_stac_collection, "r") as collection_file:
    collection = ItemCollection.from_dict(json.load(collection_file))

In [6]:
data_filenames = []
for item in collection.items:
    for asset_key in item.assets:
        asset = item.assets[asset_key]
        data_filenames.append(asset.href)

# Summary Table

AKA the result of this "application"

In [7]:
column_names = [
    'product_name',
    'product_name_type_id',
    'shortname',
    'product_version',
    'date_created',
    'time_coverage_start',
    'time_coverage_end',
    'geospatial_lat_mid',
    'geospatial_lon_mid',
]

In [8]:
table_data = []
for data_file in data_filenames:
    nc_file = netCDF4.Dataset(data_file, "r")
    
    table_row = []
    for col_name in column_names:
        table_row.append(getattr(nc_file, col_name))
    table_data.append(table_row)

In [9]:
tabulate(table_data, headers=column_names, tablefmt='html')

product_name,product_name_type_id,shortname,product_version,date_created,time_coverage_start,time_coverage_end,geospatial_lat_mid,geospatial_lon_mid
SNDR.SS1330.CHIRP.20160822T0005.m06.g001.L1_AQ.std.v02_48.G.200425095850.nc,L1_AQ,SNDR13CHRP1,v02.48.00,2021-04-25T05:59:08Z,2016-08-22T00:05:22Z,2016-08-22T00:11:22Z,-48.6062,12.4563
SNDR.SS1330.CHIRP.20160822T0011.m06.g002.L1_AQ.std.v02_48.G.200425095901.nc,L1_AQ,SNDR13CHRP1,v02.48.00,2021-04-25T05:59:19Z,2016-08-22T00:11:22Z,2016-08-22T00:17:22Z,-69.3979,-1.98753


In [10]:
with open(summary_table_filename, "w") as summary_file:
    summary_file.write(tabulate(table_data, headers=column_names))

# Create stage-out item collection

In [11]:
output_item = Item(
    id=os.path.splitext(summary_table_filename)[0],
    geometry="",
    bbox="",
    datetime=datetime.now(timezone.utc),
    properties={
        "datetime": datetime.now(timezone.utc).isoformat().replace('+00:00', 'Z'),
        "start_datetime": datetime.now(timezone.utc).isoformat().replace('+00:00', 'Z'),
        "end_datetime": datetime.now(timezone.utc).isoformat().replace('+00:00', 'Z'),
        "created": datetime.now(timezone.utc).isoformat().replace('+00:00', 'Z'),
        "updated": datetime.now(timezone.utc).isoformat().replace('+00:00', 'Z'),
    },
    assets={
        "data": Asset(href=summary_table_filename,title="Main Data File"),
    }
)

output_collection = ItemCollection([output_item])

In [12]:
with open(output_stac_collection, "w") as collection_file:
    json.dump(output_collection.to_dict(), collection_file)